In [83]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import json

from selenium.webdriver.support.ui import Select, WebDriverWait

def get_champion_data(url):
    driver = webdriver.Chrome('/Users/user/Downloads/chromedriver')
    driver.get(url)
    champ_url = driver.current_url.split('championstats')[0]
    name = find_between(champ_url, "https://lolcounter.gg/", "/")
    folder = 'champstats/'+name
    print("starting ", name)
    select = driver.find_elements_by_tag_name("fieldset")  #get the list of leagues & the list of lanes played
    league_options = select[0].find_elements_by_tag_name("li") #get all list elements inside leagues (platplus, plat, etc.)
    lane_options = select[1].find_elements_by_tag_name("li") #get all list elements inside lanes (mid, top, etc.)

    optionsList = {'lane': [], 'league': []}

    for option in league_options: #iterate over the options, place attribute of onclick in list
        optionsList['league'].append(option.get_attribute("onclick"))

    for option in lane_options: #iterate over the options, place attribute of onclick in list
        optionsList['lane'].append(option.get_attribute("onclick"))

    championJson = {}
    for optionHref in optionsList['league']:
        optionValue = 'platplus'
        if "championstats.php?" not in optionHref:
            optionValue = find_between(optionHref, "championstats", ".php?")
            link = champ_url + find_between(optionHref, "location.href='", "';")
            print(link)
            driver.get(link)
        for laneHref in optionsList['lane']:
            lane = find_between(laneHref, "Lane=", "';")
            link = champ_url + find_between(optionHref, "location.href='", "?") + find_between(laneHref, "hp", "';")
            print(link)
            driver.get(link)
            soup = BeautifulSoup(driver.page_source,'lxml')

            print("starting loop on option {} in lane {}".format(optionValue, lane))

            skill_order = driver.find_element_by_class_name("Skill-Order2")    #get highest win % skill order
            skill_order_table = skill_order.find_element_by_class_name("skill-order")
            skills = skill_order_table.find_elements_by_class_name("skill")
            counter = -1
            skill_images = []
            skill_order = [0] * 18
            for skill in skills:
                if counter == -1:
                    counter += 1
                    continue
                # skill has img and skill selections
                img = skill.find_element_by_tag_name("img") #deal with img first
                img_src = img.get_attribute("src")
                skill_images.append(img_src)
                skills_selection = skill.find_element_by_class_name("skill-selections")
                skills_selection_list = skills_selection.find_elements_by_tag_name("div")
                skill_counter = 0
                current_skill = 'qwer'[counter]
                for skill_selection in skills_selection_list:
                    text = skill_selection.text.replace(" ", "")
                    if text:
                        skill_order[skill_counter] = current_skill
                    skill_counter += 1
                counter += 1

            if skill_order == [0] * 18:
                continue
            if optionValue not in championJson:
                championJson[optionValue] = {}
            if lane not in championJson[optionValue]:
                championJson[optionValue][lane] = {}
            if "skillImages" not in championJson:
                championJson["skillImages"] = skill_images
            championJson[optionValue][lane]["skill_order"] = skill_order
            #print(skill_images)
            #print(skill_order)

            table = soup.find_all('table')[0] # get stats table
            df = pd.read_html(str(table))[0] # add that to df
            stats_list = json.loads(df.to_json(orient='records'))
            stats_dict = {}
            for stat in stats_list:
                type_stat = stat["Type"]
                ave = stat["Average"]
                stat.clear()
                stats_dict[type_stat] = ave
            championJson[optionValue][lane]["stats"] = stats_dict
            #print(df)
    
    driver.close()
    filename = folder + '.json'
    with open(filename, 'w') as f:
         json.dump(championJson, f)
    print("success! ", name)

In [23]:
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

find_between("location.href='championstatsplat.php?Lane=TOP';", "hp", "';")

'?Lane=TOP'

In [84]:
def get_champ_links():
    links = []
    url = 'https://lolcounter.gg/index.php'
    driver = webdriver.Chrome('/Users/user/Downloads/chromedriver')
    driver.get(url)
    host = "https://lolcounter.gg/"
    champ_stats = '/championstats.php?Lane=TOP'
    champion_table = driver.find_element_by_id("champion-list")
    elements_list = champion_table.find_elements_by_class_name("inlinechamps")
    for champ in elements_list:
        name = champ.get_attribute("data-champion-name").replace(" ", "%20")
        link = host + name + champ_stats
        links.append(link)
    driver.close()
    return links
champ_links = get_champ_links()
for link in champ_links:
    print(link)
    get_champion_data(link)

https://lolcounter.gg/aatrox/championstats.php?Lane=TOP
starting  aatrox
https://lolcounter.gg/aatrox/championstats.php?Lane=TOP
starting loop on option platplus in lane TOP
https://lolcounter.gg/aatrox/championstats.php?Lane=JUNGLE
starting loop on option platplus in lane JUNGLE
https://lolcounter.gg/aatrox/championstatsplat.php?Lane=TOP
https://lolcounter.gg/aatrox/championstatsplat.php?Lane=TOP
starting loop on option plat in lane TOP
https://lolcounter.gg/aatrox/championstatsplat.php?Lane=JUNGLE
starting loop on option plat in lane JUNGLE
https://lolcounter.gg/aatrox/championstatsgold.php?Lane=TOP
https://lolcounter.gg/aatrox/championstatsgold.php?Lane=TOP
starting loop on option gold in lane TOP
https://lolcounter.gg/aatrox/championstatsgold.php?Lane=JUNGLE
starting loop on option gold in lane JUNGLE
https://lolcounter.gg/aatrox/championstatssilver.php?Lane=TOP
https://lolcounter.gg/aatrox/championstatssilver.php?Lane=TOP
starting loop on option silver in lane TOP
https://lolcount

https://lolcounter.gg/ashe/championstatssilver.php?Lane=ADC
https://lolcounter.gg/ashe/championstatssilver.php?Lane=ADC
starting loop on option silver in lane ADC
https://lolcounter.gg/ashe/championstatsbronze.php?Lane=ADC
https://lolcounter.gg/ashe/championstatsbronze.php?Lane=ADC
starting loop on option bronze in lane ADC
success!  ashe
https://lolcounter.gg/aurelion%20sol/championstats.php?Lane=TOP
starting  aurelion%20sol
https://lolcounter.gg/aurelion%20sol/championstats.php?Lane=MIDDLE
starting loop on option platplus in lane MIDDLE
https://lolcounter.gg/aurelion%20sol/championstats.php?Lane=JUNGLE
starting loop on option platplus in lane JUNGLE
https://lolcounter.gg/aurelion%20sol/championstatsplat.php?Lane=MIDDLE
https://lolcounter.gg/aurelion%20sol/championstatsplat.php?Lane=MIDDLE
starting loop on option plat in lane MIDDLE
https://lolcounter.gg/aurelion%20sol/championstatsplat.php?Lane=JUNGLE
starting loop on option plat in lane JUNGLE
https://lolcounter.gg/aurelion%20sol/ch

starting loop on option gold in lane TOP
https://lolcounter.gg/camille/championstatssilver.php?Lane=TOP
https://lolcounter.gg/camille/championstatssilver.php?Lane=TOP
starting loop on option silver in lane TOP
https://lolcounter.gg/camille/championstatsbronze.php?Lane=TOP
https://lolcounter.gg/camille/championstatsbronze.php?Lane=TOP
starting loop on option bronze in lane TOP
success!  camille
https://lolcounter.gg/cassiopeia/championstats.php?Lane=TOP
starting  cassiopeia
https://lolcounter.gg/cassiopeia/championstats.php?Lane=MIDDLE
starting loop on option platplus in lane MIDDLE
https://lolcounter.gg/cassiopeia/championstatsplat.php?Lane=MIDDLE
https://lolcounter.gg/cassiopeia/championstatsplat.php?Lane=MIDDLE
starting loop on option plat in lane MIDDLE
https://lolcounter.gg/cassiopeia/championstatsgold.php?Lane=MIDDLE
https://lolcounter.gg/cassiopeia/championstatsgold.php?Lane=MIDDLE
starting loop on option gold in lane MIDDLE
https://lolcounter.gg/cassiopeia/championstatssilver.ph

https://lolcounter.gg/draven/championstatsgold.php?Lane=ADC
https://lolcounter.gg/draven/championstatsgold.php?Lane=ADC
starting loop on option gold in lane ADC
https://lolcounter.gg/draven/championstatssilver.php?Lane=ADC
https://lolcounter.gg/draven/championstatssilver.php?Lane=ADC
starting loop on option silver in lane ADC
https://lolcounter.gg/draven/championstatsbronze.php?Lane=ADC
https://lolcounter.gg/draven/championstatsbronze.php?Lane=ADC
starting loop on option bronze in lane ADC
success!  draven
https://lolcounter.gg/ekko/championstats.php?Lane=TOP
starting  ekko
https://lolcounter.gg/ekko/championstats.php?Lane=MIDDLE
starting loop on option platplus in lane MIDDLE
https://lolcounter.gg/ekko/championstats.php?Lane=TOP
starting loop on option platplus in lane TOP
https://lolcounter.gg/ekko/championstatsplat.php?Lane=MIDDLE
https://lolcounter.gg/ekko/championstatsplat.php?Lane=MIDDLE
starting loop on option plat in lane MIDDLE
https://lolcounter.gg/ekko/championstatsplat.php?

https://lolcounter.gg/fizz/championstatsplat.php?Lane=MIDDLE
starting loop on option plat in lane MIDDLE
https://lolcounter.gg/fizz/championstatsgold.php?Lane=MIDDLE
https://lolcounter.gg/fizz/championstatsgold.php?Lane=MIDDLE
starting loop on option gold in lane MIDDLE
https://lolcounter.gg/fizz/championstatssilver.php?Lane=MIDDLE
https://lolcounter.gg/fizz/championstatssilver.php?Lane=MIDDLE
starting loop on option silver in lane MIDDLE
https://lolcounter.gg/fizz/championstatsbronze.php?Lane=MIDDLE
https://lolcounter.gg/fizz/championstatsbronze.php?Lane=MIDDLE
starting loop on option bronze in lane MIDDLE
success!  fizz
https://lolcounter.gg/galio/championstats.php?Lane=TOP
starting  galio
https://lolcounter.gg/galio/championstats.php?Lane=TOP
starting loop on option platplus in lane TOP
https://lolcounter.gg/galio/championstats.php?Lane=MIDDLE
starting loop on option platplus in lane MIDDLE
https://lolcounter.gg/galio/championstatsplat.php?Lane=TOP
https://lolcounter.gg/galio/champi

...

https://lolcounter.gg/tahm%20kench/championstatssilver.php?Lane=TOP
starting loop on option silver in lane TOP
https://lolcounter.gg/tahm%20kench/championstatsbronze.php?Lane=SUPPORT
https://lolcounter.gg/tahm%20kench/championstatsbronze.php?Lane=SUPPORT
starting loop on option bronze in lane SUPPORT
https://lolcounter.gg/tahm%20kench/championstatsbronze.php?Lane=TOP
starting loop on option bronze in lane TOP
success!  tahm%20kench
https://lolcounter.gg/taliyah/championstats.php?Lane=TOP
starting  taliyah
https://lolcounter.gg/taliyah/championstats.php?Lane=MIDDLE
starting loop on option platplus in lane MIDDLE
https://lolcounter.gg/taliyah/championstatsplat.php?Lane=MIDDLE
https://lolcounter.gg/taliyah/championstatsplat.php?Lane=MIDDLE
starting loop on option plat in lane MIDDLE
https://lolcounter.gg/taliyah/championstatsgold.php?Lane=MIDDLE
https://lolcounter.gg/taliyah/championstatsgold.php?Lane=MIDDLE
starting loop on option gold in lane MIDDLE
https://lolcounter.gg/taliyah/champio

starting  tryndamere
https://lolcounter.gg/tryndamere/championstats.php?Lane=TOP
starting loop on option platplus in lane TOP
https://lolcounter.gg/tryndamere/championstats.php?Lane=JUNGLE
starting loop on option platplus in lane JUNGLE
https://lolcounter.gg/tryndamere/championstatsplat.php?Lane=TOP
https://lolcounter.gg/tryndamere/championstatsplat.php?Lane=TOP
starting loop on option plat in lane TOP
https://lolcounter.gg/tryndamere/championstatsplat.php?Lane=JUNGLE
starting loop on option plat in lane JUNGLE
https://lolcounter.gg/tryndamere/championstatsgold.php?Lane=TOP
https://lolcounter.gg/tryndamere/championstatsgold.php?Lane=TOP
starting loop on option gold in lane TOP
https://lolcounter.gg/tryndamere/championstatsgold.php?Lane=JUNGLE
starting loop on option gold in lane JUNGLE
https://lolcounter.gg/tryndamere/championstatssilver.php?Lane=TOP
https://lolcounter.gg/tryndamere/championstatssilver.php?Lane=TOP
starting loop on option silver in lane TOP
https://lolcounter.gg/trynda

https://lolcounter.gg/velkoz/championstats.php?Lane=SUPPORT
starting loop on option platplus in lane SUPPORT
https://lolcounter.gg/velkoz/championstatsplat.php?Lane=MIDDLE
https://lolcounter.gg/velkoz/championstatsplat.php?Lane=MIDDLE
starting loop on option plat in lane MIDDLE
https://lolcounter.gg/velkoz/championstatsplat.php?Lane=SUPPORT
starting loop on option plat in lane SUPPORT
https://lolcounter.gg/velkoz/championstatsgold.php?Lane=MIDDLE
https://lolcounter.gg/velkoz/championstatsgold.php?Lane=MIDDLE
starting loop on option gold in lane MIDDLE
https://lolcounter.gg/velkoz/championstatsgold.php?Lane=SUPPORT
starting loop on option gold in lane SUPPORT
https://lolcounter.gg/velkoz/championstatssilver.php?Lane=MIDDLE
https://lolcounter.gg/velkoz/championstatssilver.php?Lane=MIDDLE
starting loop on option silver in lane MIDDLE
https://lolcounter.gg/velkoz/championstatssilver.php?Lane=SUPPORT
starting loop on option silver in lane SUPPORT
https://lolcounter.gg/velkoz/championstatsbr

starting  xayah
https://lolcounter.gg/xayah/championstats.php?Lane=ADC
starting loop on option platplus in lane ADC
https://lolcounter.gg/xayah/championstatsplat.php?Lane=ADC
https://lolcounter.gg/xayah/championstatsplat.php?Lane=ADC
starting loop on option plat in lane ADC
https://lolcounter.gg/xayah/championstatsgold.php?Lane=ADC
https://lolcounter.gg/xayah/championstatsgold.php?Lane=ADC
starting loop on option gold in lane ADC
https://lolcounter.gg/xayah/championstatssilver.php?Lane=ADC
https://lolcounter.gg/xayah/championstatssilver.php?Lane=ADC
starting loop on option silver in lane ADC
https://lolcounter.gg/xayah/championstatsbronze.php?Lane=ADC
https://lolcounter.gg/xayah/championstatsbronze.php?Lane=ADC
starting loop on option bronze in lane ADC
success!  xayah
https://lolcounter.gg/xerath/championstats.php?Lane=TOP
starting  xerath
https://lolcounter.gg/xerath/championstats.php?Lane=MIDDLE
starting loop on option platplus in lane MIDDLE
https://lolcounter.gg/xerath/championsta

starting loop on option platplus in lane MIDDLE
https://lolcounter.gg/zilean/championstatsplat.php?Lane=SUPPORT
https://lolcounter.gg/zilean/championstatsplat.php?Lane=SUPPORT
starting loop on option plat in lane SUPPORT
https://lolcounter.gg/zilean/championstatsplat.php?Lane=MIDDLE
starting loop on option plat in lane MIDDLE
https://lolcounter.gg/zilean/championstatsgold.php?Lane=SUPPORT
https://lolcounter.gg/zilean/championstatsgold.php?Lane=SUPPORT
starting loop on option gold in lane SUPPORT
https://lolcounter.gg/zilean/championstatsgold.php?Lane=MIDDLE
starting loop on option gold in lane MIDDLE
https://lolcounter.gg/zilean/championstatssilver.php?Lane=SUPPORT
https://lolcounter.gg/zilean/championstatssilver.php?Lane=SUPPORT
starting loop on option silver in lane SUPPORT
https://lolcounter.gg/zilean/championstatssilver.php?Lane=MIDDLE
starting loop on option silver in lane MIDDLE
https://lolcounter.gg/zilean/championstatsbronze.php?Lane=SUPPORT
https://lolcounter.gg/zilean/champio